In [1]:
MODEL = "ollama:qwen2.5:1.5b"

In [2]:
import functools
import json


def log_function_call(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        print(f"Calling {func.__name__} with: {kwargs}")
        result = func(*args, **kwargs)
        print(f"Function {func.__name__} returned: {json.dumps(result, indent=2)}")
        return result

    return wrapper


## Function Calling (Tools)

Function calling allows the LLM to use external tools. You can define a set of functions and the model will intelligently decide when to use them based on the user's prompt. This is a powerful way to extend the capabilities of the model, allowing it to interact with external APIs, databases, or any other system.


In [3]:
import requests
import time
from pydantic_ai import Agent


@log_function_call
def get_weather(latitude: float, longitude: float) -> dict:
    """Get current weather data for a specific location."""
    url = (
        "https://api.open-meteo.com/v1/forecast"
        f"?latitude={latitude}&longitude={longitude}"
        "&current=temperature_2m,wind_speed_10m"
    )
    response = requests.get(url)
    return response.json()


agent = Agent(
    MODEL,
    tools=[get_weather],
    system_prompt=f"You are a helpful assistant that can use tools to get information. Current time is {time.time()}",
)

In [4]:
result = await agent.run(f"How windy is it in Warsaw today?")
print(result.output, end="\n\n")

Calling get_weather with: {'latitude': 52.2345, 'longitude': 21.0204}
Function get_weather returned: {
  "latitude": 52.23009,
  "longitude": 21.017075,
  "generationtime_ms": 0.043272972106933594,
  "utc_offset_seconds": 0,
  "timezone": "GMT",
  "timezone_abbreviation": "GMT",
  "elevation": 114.0,
  "current_units": {
    "time": "iso8601",
    "interval": "seconds",
    "temperature_2m": "\u00b0C",
    "wind_speed_10m": "km/h"
  },
  "current": {
    "time": "2026-01-14T18:00",
    "interval": 900,
    "temperature_2m": -3.2,
    "wind_speed_10m": 15.5
  }
}
Unfortunately, I don't have the current date in my system but according to the response, on January 14th (a possible today), Warsaw's wind speed was 15.5 km/h at 6 PM local time. Please note that this is just an approximation based on historical data and might not reflect the current weather situation. For the most accurate and up-to-date information I recommend checking a reliable weather forecasting website, such as AccuWeath

## Saving State from Tools*

Sometimes you want to persist data that tools collect during execution. PydanticAI allows you to pass a state object (`deps`) through the agent's context, which tools can read from and write to.

In [5]:
from pydantic import BaseModel
from pydantic_ai import Agent, RunContext
import requests
from typing import Optional


class WeatherState(BaseModel):
    last_temperature: Optional[float] = None
    last_location: Optional[str] = None


@log_function_call
def get_weather_with_state(ctx: RunContext[WeatherState], latitude: float, longitude: float, location_name: str) -> str:
    """Get weather and save the temperature to state."""
    url = (
        "https://api.open-meteo.com/v1/forecast"
        f"?latitude={latitude}&longitude={longitude}"
        "&current=temperature_2m,wind_speed_10m"
    )
    response = requests.get(url)
    data = response.json()

    temperature = data["current"]["temperature_2m"]

    ctx.deps.last_temperature = temperature
    ctx.deps.last_location = location_name

    return f"Temperature in {location_name}: {temperature}°C"


agent = Agent(
    MODEL,
    deps_type=WeatherState,
    tools=[get_weather_with_state],
    system_prompt="You are a helpful weather assistant."
)

In [6]:
state = WeatherState()
result = await agent.run(
    "What's the temperature in Warsaw? Coordinates: 52.23, 21.01",
    deps=state
)

print("Agent response:", result.output)
print(f"\nSaved to state:")
print(f"  Location: {state.last_location}")
print(f"  Temperature: {state.last_temperature}°C")


Calling get_weather_with_state with: {'latitude': 52.23, 'longitude': 21.01, 'location_name': 'Warsaw'}
Function get_weather_with_state returned: "Temperature in Warsaw: -3.2\u00b0C"
Agent response: The current temperature in Warsaw is -3.2°C (27.7°F).

Saved to state:
  Location: Warsaw
  Temperature: -3.2°C


---

### Real-World Examples

**1. Healthcare - 3M Health Information Systems**
- Integrated AI with AWS to improve clinical documentation
- AI listens to doctor-patient conversations, accesses patient records, and assists in completing notes
- Reduces administrative workload and minimizes manual entry errors
- Source: [3M AWS Collaboration](https://www.simbo.ai/blog/leveraging-function-calling-in-large-language-models)

**2. Healthcare - GE Healthcare Edison Platform**
- Combines analytics, ML, and data from medical devices and hospitals
- AI agents suggest adjustments to patient care based on real-time data
- Provides operational insights and clinical decision support
- Source: [GE Healthcare Edison](https://www.simbo.ai/blog/leveraging-function-calling-in-large-language-models)

**3. Developer Platforms - Vercel & Telnyx**
- Vercel: Uses OpenAI function calling for AI model deployment and monitoring
- Telnyx: Offers API with function calling for customer service bots and data analysis
- Sources: [Vercel](https://www.appsruntheworld.com/customers-database/products/view/openai-function-calling), [Telnyx](https://telnyx.com/resources/function-calling-ai-overview)

---

## 🎯 Task 1: Attach an Existing Tool

**Objective**: A `get_city_population` function is provided below. Your job is to attach it to the agent's tools.

**Instructions**:
- Review the `get_city_population` function below (it's already complete)
- Add it to the agent's `tools` list
- Test it by asking the agent about a city's population


In [8]:
import requests
from pydantic_ai import Agent


@log_function_call
def get_city_population(city: str) -> dict:
    """Get population data for a specific city using the API Ninjas City API.

    Args:
        city: Name of the city

    Returns:
        Dictionary with city information including population
    """
    mock_data = {
        "Warsaw": {"name": "Warsaw", "population": 1863056},
        "Krakow": {"name": "Krakow", "population": 779115},
        "New York": {"name": "New York", "population": 8336817},
        "London": {"name": "London", "population": 9002488},
    }

    city_data = mock_data.get(city, {"name": city, "population": "Unknown"})
    return city_data


# TODO: Add get_city_population to the agent below
agent = Agent(
    MODEL,
    system_prompt="You are a helpful assistant that can provide city information.",
    tools=[get_city_population],
)

result = await agent.run("What is the population of Warsaw in Poland?")
print(result.output)


Calling get_city_population with: {'city': 'Warsaw'}
Function get_city_population returned: {
  "name": "Warsaw",
  "population": 1863056
}
The population of Warsaw, Poland is approximately 1.863 million people.


## 🎯 Task 2: Create Your Own Tool*

**Objective**: Create a `roll_dice` function that simulates rolling dice for games.

**Instructions**:
- Implement the `roll_dice` function below
- The function should accept `num_dice` (number of dice) and `sides` (number of sides per die)
- Use Python's `random` module to generate random numbers
- Return a dictionary with the individual rolls and the total
- Register your function with the agent and test it


In [13]:
import random
from pydantic_ai import Agent


@log_function_call
def roll_dice(num_dice: int, sides: int = 6) -> dict:
    """Roll dice and return the results.

    Args:
        num_dice: Number of dice to roll
        sides: Number of sides on each die (default: 6)

    Returns:
        Dictionary with 'rolls' (list of individual results) and 'total' (sum)
    """
    # TODO: Implement this function
    # Hint: Use random.randint() to roll each die
    # Return a dict like: {"rolls": [3, 5, 2], "total": 10}
    rolls = [random.randint(1, sides) for _ in range(num_dice)]
    return {
        "rolls": rolls,
        "total": sum(rolls),
    }


# TODO: Create an agent with roll_dice as a tool
agent = Agent(
    MODEL,
    system_prompt="You are a helpful assistant that can roll dice for games and tabletop RPGs.",
    tools=[roll_dice],
)

result = await agent.run("Roll 3 six-sided dice for me")
print(result.output)


Calling roll_dice with: {'num_dice': 3, 'sides': 6}
Function roll_dice returned: {
  "rolls": [
    6,
    2,
    6
  ],
  "total": 14
}
The results of the roll are:

* Die 1: 6
* Die 2: 2
* Die 3: 6

The total is: 14
